In [40]:
# import section

import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline 

Importing the full GSS datafile over all years does not work, so we needed to import every year separately. The files of all available years are in one directory on our computers, and LoadData function below loops through every year in the directory and appends it to the total data file. Initially we worked in Spyder as we could collaborate using Github, but there we had to load the data everytime we ran the full program, which took a lot of time. We made a separate data loading program before we realized that Jupyter works with blocks. Luckily we could incorporate some of our previous code, and we now work on this notebook through Github, although the updated file has to be uploaded to the server each time. 

In [2]:
# Data loading function

def LoadData():
    year = 1972
    total_data = {}
    years = []
    for i in range(28):
        if year == 1979 or year == 1981 or year == 1984 or year == 1986 or year ==1992:
            year += 1
        years.append(str(year))
        print year,
        data = pd.io.stata.read_stata("/Users/Matthijsmeijers/Documents/MasterclassDATA/GSSdataproject/MasterclassBDProject/GSS" + str(year) + ".DTA")
        total_data[str(year)] = data
        year += 1
        if year > 1994:
            year += 1
        
    print "Data Loaded"
    return total_data

total_data = LoadData()

1972 1973 1974 1975 1976 1977 1978 1980 1982 1983 1985 1987 1988 1989 1990 1991 1993 1994 1996 1998 2000 2002 2004 2006 2008 2010 2012 2014 Data Loaded


After having loaded the data, we noticed that the answers to the survey questions are not constant over the years. In for instance the category religion, some additional religions are added over the years. In order for us to be able to analyse the data properly, we decided that it was necessary to add all possible answers to each category for every year. The AllAnswers function does this for a specified category.

In [21]:
# Returns a list of all possible answers in one category over all years
def AllAnswers(category):
    answers_cat = []
    for year in total_data.keys():
        if category not in total_data[year].keys():
            continue
        else:
            cat = getattr(total_data[year], category)
            for answer in cat:
                if type(answer) == str:
                    answer = answer.lower()
                if answer not in answers_cat:
                    answers_cat.append(answer)
                    
    return answers_cat

The block below is dedicated to calculating proportions of answers within a certain category in one year. This is useful for getting a general idea of distributions within a single year. The first function, Count, counts the frequencies of the answers to a survey question (category), and returns them in a dictionary. The second function, CategoryCompletion, appends the answers that were not chosen or included in the options for a survey question of a single year to the output of the Counted function with a very small value (such that entropy calculations are not affected). The last function, Proportions, transforms the frequencies into proportions and returns them in a list.

In [30]:
# Counting function
# Returns dictionary with frequency of all answers in a chosen category
def Count(year, category):
    if category not in total_data[year].keys():
        return "Error: category nonexistent"
    else:
        filtered = getattr(total_data[year], category)
        counted = {}
        for answer in filtered:
            if type(answer) == str:
                answer = answer.lower()
            if answer not in counted:
                counted[answer] = 1
            else:
                counted[answer] = counted[answer] + 1   
        return counted

# Returns completed dictionary of answers to some categories with keys answers and values counts. Non-existent answers
# in some year are given count = 1e-16
def CategoryCompletion(year, category):
    frequency = Count(year, category)
    if type(frequency) == str:
        return frequency
    allanswers = AllAnswers(category)
    for answer in allanswers:
        if answer not in frequency.keys():
            frequency[answer] = 1e-16
    return frequency
    
# Proportion function
# Returns list with proportions based on counting function
def Proportions(year, category):  # input should be output of Count function
    frequencies = CategoryCompletion(year, category)
    if type(frequencies) == str:
        return frequencies
    numbervals = frequencies.values()
    population = 0
    proportions = []
    for frequency in numbervals:
        population += frequency
    for frequency in numbervals:
        proportions.append(float(frequency) / float(population))
        
    return proportions

While the block above focuses on single categories, it is statistically speaking more useful to look at correlations between proportions within categories. The block below performs this action by calculating the Kullback-Leibler divergence between the general opinion on a certain subject and the opinion of a subgroup on a certain subject (for example, what is the divergence between the opinion on premarital sex of protestants and the opinion of the whole population on that topic?). The block is largely similar to the block above, but the functions CrossCount, CrossCategoryComplete, and CrossProportions only take the answers of a subset of the population to a certain question into account (in code: total_data["1990", premarsx][religion == "protestant"]).

In [35]:
# Returns a dictionary of all answers to 'category1' in 'year' within subcategory of category2     
def CrossCount(year, category1, category2, subcategory):
    if category1 not in total_data[year].keys():
        return "Error: category 1 nonexistent"
    elif category2 not in total_data[year].keys():
        return "Error: category 2 & subcategory nonexistent"
    else:
        filtered = getattr(total_data[year], category1)[(getattr(total_data[year], category2)) == subcategory]
        counted = {}
        for answer in filtered:
            if type(answer) == str:
                answer = answer.lower()
            if answer not in counted:
                counted[answer] = 1
            else:
                counted[answer] += 1   
        return counted
# returns completed dictionary from crosscount
def CrossCategoryComplete(year, category1, category2, subcategory):
    frequencies = CrossCount(year, category1, category2, subcategory)
    totalcats = AllAnswers(category1)
    if type(frequencies) == str:
        return frequencies
    for category in totalcats:
        if category not in frequencies.keys():
            frequencies[category] = 1e-16
    return frequencies

# Returns list with proportions of answers to category1 within subcategory of category2
def CrossProportions(year, category1, category2, subcategory):
    counted = CrossCategoryComplete(year, category1, category2, subcategory)
    if type(counted) == str:
        return counted
    numbervals = counted.values()
    totalval = 0
    proportions = []
    for item in numbervals:
        totalval += item
    for item in numbervals:
        proportions.append(float(item) / float(totalval))
    return proportions

# Returns the Kullback-Leibler divergence between the answers to category1 of the total population and 
# subcategory of category 2
def Entropy(year, category1, category2, subcategory):
    dist1 = CrossProportions(year, category1, category2, subcategory)
    if type(dist1) == str:
        return dist1
    dist2 = Proportions(year, category1)
    if len(dist1) != len(dist2):
        params = [dist1, dist2]
        lengths = [len(dist1), len(dist2)]
        maxlength = max(lengths)    
        for param in params:
            if len(param) < maxlength:
                for i in range(maxlength - len(param)):
                    param.append(1e-16)
    entropy = sp.stats.entropy(dist1, dist2)
    return entropy

In [100]:
ListofCategories = ['wrkstat','agewed','divorce','sibs','childs','degree','hompop','partyid',
                    'natdrug','natarms','spkath','spkrac','spkcom','spkhomo','cappun','gunlaw','courts','grass']
# Returns a list of tuples of the "number" maximal Kullback-Leibler divergences between the total population and 
# subcategory of category2 for a list of different categories (questions)
def ListofEntropies(ListofCategories, category2, subcategory, number):
    Entropylist = []
    maxentropies = []
    #Loop over categories
    for category1 in ListofCategories:
        entropies = []
        #Loop over years
        for year in total_data.keys():            
            if category1 not in total_data[str(year)].keys():
                print category1 + " not in " + str(year)
                continue
            if category2 not in total_data[str(year)].keys():
                print category2 + " not in " + str(year)
                continue
            entropies.append(Entropy(str(year), category1, category2, subcategory))
        
        Entropylist.append((np.mean(entropies), category1))
    for i in range(number - 1):
        maxentropy = max(Entropylist)
        Entropylist.remove(maxentropy)
        maxentropies.append(maxentropy)
    return maxentropies

In [95]:
# Returns a list of distributions of the population that has some religion (belief), accumulating all smaller religions
# in others
def Religionsdist(total_data, belief, others):
    Distribution = []
    for year in total_data.keys(): 
        believers = 0
        population = 0
        for believer in getattr(total_data[str(year)], 'relig'):
            if type(believer) == str:
                believer = believer.lower()
            if believer == belief:
                believers += 1
            if belief == 'other':
                if believer in others:
                    believers +=1
            if believer != 0:
                population += 1
        Distribution.append(float(believers)/float(population))
    return Distribution

# Makes plots of distributions of the population that has the belief in list 'beliefs'
def Religionplots(total_data, beliefs, others):
    Numberofbeliefs = len(beliefs)
    # Create list of years
    years = []
    for key in total_data.keys():
        years.append(int(key))
    years.sort()
    for i, belief in enumerate(beliefs):
        Distribution = Religionsdist(total_data, belief, others)
        plt.figure(1,(8,25))
        plt.subplot(Numberofbeliefs,1,i+1)
        plt.plot(years, Distribution)
        plt.ylabel("Probability")
        plt.xlabel("Year")
        plt.title(belief)
        plt.tight_layout()
        
# Returns a list of distributions of the total population that gives 'answer' to question 'category1'
def distributiontotalpop(total_data, category1, askedanswer):
    Distribution = []
    for year in total_data.keys():
        positive = 0
        if category1 not in total_data[year]:
            continue
        for answer in getattr(total_data[str(year)], category1):
            if type(answer) == str:
                answer = answer.lower()
            if answer == askedanswer:
                positive += 1
        Distribution.append(positive/float(len(getattr(total_data[str(year)], category1))))
    return Distribution

# Returns a list of distributions of a constricted population that gives 'answer' to question 'category1'
def distributionpartialpop(total_data,category1, askedanswer, category2, subcategory):
    Distribution = []
    for year in total_data.keys():
        positive = 0
        if category1 not in total_data[year][getattr(total_data[year], category2) == subcategory]:
            continue
        for answer in getattr(total_data[str(year)], category1)[getattr(total_data[str(year)], category2) == subcategory]:
            if type(answer) == str:
                answer = answer.lower()
            if answer == askedanswer:
                positive += 1
        Distribution.append(positive/float(len(getattr(total_data[str(year)],category1)[getattr(total_data[str(year)], category2) == subcategory])))
    return Distribution
    


    
                    

## 

In [101]:
others = ['nan', 'buddhism', 'hinduism', 'inter-nondenominational', 'moslem/islam', 'native american', 
          'orthodox-christian', 'other eastern', 'christian']
#plt.plot(distributiontotalpop(total_data, 'partyid', 'strong republican'))
#plt.plot(distributiontotalpop(total_data, 'grass', 'not legal'))

#np.unique(total_data['2014'].partyid)
#Religionplots(total_data, ['protestant','catholic','none','jewish', 'other'], others)
#Proportions('1985', 'relig')
ListofEntropies(ListofCategories, 'relig', 'catholic', 4)

agewed not in 1996
agewed not in 2014
agewed not in 2010
agewed not in 2012
agewed not in 2002
agewed not in 2000
agewed not in 2004
agewed not in 1998
agewed not in 2008
natdrug not in 1972
natarms not in 1972
spkath not in 1983
spkath not in 1978
spkath not in 1975
spkrac not in 1983
spkrac not in 1978
spkrac not in 1975
spkrac not in 1974
spkrac not in 1973
spkrac not in 1972
spkcom not in 1983
spkcom not in 1978
spkcom not in 1975
spkhomo not in 1983
spkhomo not in 1978
spkhomo not in 1975
spkhomo not in 1972
cappun not in 1973
cappun not in 1972
gunlaw not in 1983
gunlaw not in 1978
grass not in 1985
grass not in 1982
grass not in 1977
grass not in 1974
grass not in 1972


[(8.5637007920445196, 'agewed'),
 (0.58504933157862749, 'sibs'),
 (0.026488120969954874, 'degree')]